
<a id='chap-tpacp'></a>

# This work was done by:

    ELGHABI Taha
    EL Batouri Badr-eddine

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## ACP des données sur le sommeil des mammifères

L’ACP sera d’abord appliquée aux données concernant le sommeil des mammifères (ci joint le fichier csv mammals.csv).

In [3]:
data=pd.read_csv("mammals.csv", delimiter=';', usecols=[1,2,3,4,5,6,7,8,9,10])#, skiprows=1

In [4]:
data

,BodyW,BrainW,SWS,PS,TS,LifeSpan,GT,PI,SEI,ODI
0,6654.000,5712.0,8.35,1.8,3.30,38.6,645.0,3,5,3
1,1.000,6.6,6.30,2.0,8.30,4.5,42.0,3,1,3
2,3.385,44.5,8.35,1.8,12.50,14.0,60.0,1,1,1
3,0.920,5.7,8.35,1.8,16.50,15.1,25.0,5,2,3
4,2547.000,4603.0,2.10,1.8,3.90,69.0,624.0,3,5,4
...,...,...,...,...,...,...,...,...,...,...
57,2.000,12.3,4.90,0.5,5.40,7.5,200.0,3,1,3
58,0.104,2.5,13.20,2.6,15.80,2.3,46.0,3,2,2
59,4.190,58.0,9.70,0.6,10.30,24.0,210.0,4,3,4
60,3.500,3.9,12.80,6.6,19.40,3.0,14.0,2,1,1


## Question :

Appliquez une ACP aux données de `mammals` et affichez les valeurs propres.

### Preprocessing step

    We have to scale our data before using our PCA

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data.values)

In [7]:
scaled_df = pd.DataFrame(scaled_data)

In [8]:
scaled_df

,0,1,2,3,4,5,6,7,8,9
0,7.237778,5.883379,-0.078244,-0.108003,-1.636219,1.087707,3.578151,0.088109,1.621215,0.270776
1,-0.221768,-0.299686,-0.719848,0.047722,-0.504267,-0.861297,-0.679755,0.088109,-0.891668,0.270776
2,-0.219094,-0.258613,-0.078244,-0.108003,0.446574,-0.318320,-0.552653,-1.277583,-0.891668,-1.128234
3,-0.221858,-0.300661,-0.078244,-0.108003,1.352135,-0.255448,-0.799795,1.453801,-0.263447,0.270776
4,2.632885,4.681532,-2.034353,-0.108003,-1.500385,2.825235,3.429866,0.088109,1.621215,0.970281
...,...,...,...,...,...,...,...,...,...,...
57,-0.220647,-0.293509,-1.158016,-1.120214,-1.160799,-0.689831,0.435915,0.088109,-0.891668,0.270776
58,-0.222773,-0.304129,1.439696,0.514897,1.193662,-0.987039,-0.651510,0.088109,-0.263447,-0.428729
59,-0.218191,-0.243983,0.344275,-1.042352,-0.051486,0.253236,0.506527,0.770955,0.364773,0.970281
60,-0.218965,-0.302612,1.314505,3.629393,2.008668,-0.947030,-0.877468,-0.594737,-0.891668,-1.128234



### Sklearn PCA

In [9]:
from sklearn.decomposition import PCA

In [10]:
sklearn_pca = PCA(n_components=2)

In [11]:
reduced_data = sklearn_pca.fit_transform(scaled_df.values)#ou sklearn_pca.fit_transform(scaled_data)

In [12]:
reduced_data

array([[ 6.50530263,  6.7330641 ],
       [-0.42443283, -0.80647942],
       [-1.56762174,  0.6345905 ],
       [-0.60859835, -1.12707436],
       [ 6.19361561,  4.17407302],
       [ 1.32018225, -0.99632356],
       [-3.4292    ,  1.41344843],
       [ 2.84382529, -0.69307734],
       [-1.96542103,  1.23919915],
       [-0.13357458,  1.67456411],
       [ 0.36715152, -1.59531354],
       [ 3.58409296, -1.3457594 ],
       [-1.04669831,  0.20950559],
       [ 3.17140474, -0.8098289 ],
       [-1.28244543,  0.30247268],
       [ 0.22000675,  0.14454502],
       [-1.28463223,  0.13276622],
       [-0.6515225 , -0.08946067],
       [ 0.02525931,  0.58455006],
       [-3.41813771,  1.6155924 ],
       [ 2.96781394, -0.77358455],
       [ 2.91793753, -2.18090134],
       [-2.05648618, -0.07778988],
       [ 0.22804031,  1.45558916],
       [ 1.04712846,  1.29117978],
       [-1.53982149,  0.74367027],
       [-1.26003136, -0.84114996],
       [ 0.91540347, -1.91026209],
       [ 4.21830947,

In [13]:
print(data.shape)
print(reduced_data.shape)

(62, 10)
(62, 2)


In [14]:

print("Pourcentage de variance expliquée : ")
print(sklearn_pca.explained_variance_ratio_)
print("Composantes principales : ")
print(sklearn_pca.components_)

Pourcentage de variance expliquée : 
[0.48017524 0.22078825]
Composantes principales : 
[[ 0.2507328   0.28338618 -0.32111983 -0.29060816 -0.38538159  0.24828001
   0.37336221  0.25423678  0.38127144  0.33119293]
 [ 0.4417748   0.47478054  0.09769085  0.23679916  0.09113044  0.30384009
   0.26672903 -0.42588243 -0.13611297 -0.37700212]]


Chaque ligne de `pca.components_` représente le vecteur unitaire qui donne la direction d’un axe factoriel. Le vecteur `pca.components_[i,:]` correspond à la valeur propre `pca.explained_variance_ratio_[i]`.

## Question à remetre avant la séance prochaine:

Appliquez une ACP aux données de `mammals` avec n_components>2 et affichez les valeurs propres et comparer les explained_variance_ratio_ .

### Sklearn PCA

In [15]:
sklearn_pca = PCA(n_components=4)

In [16]:
reduced_data = sklearn_pca.fit_transform(scaled_df.values)

In [17]:
reduced_data

array([[ 6.50530263e+00,  6.73306410e+00,  3.43983054e+00,
        -2.05458566e+00],
       [-4.24432831e-01, -8.06479421e-01, -4.17041871e-01,
        -1.29587063e+00],
       [-1.56762174e+00,  6.34590499e-01, -1.01722639e+00,
        -4.19560092e-01],
       [-6.08598346e-01, -1.12707436e+00,  8.96347766e-01,
         1.98295147e-01],
       [ 6.19361561e+00,  4.17407302e+00,  3.62007916e-01,
         7.28454239e-03],
       [ 1.32018225e+00, -9.96323557e-01,  2.93588898e-01,
         9.07090512e-01],
       [-3.42920000e+00,  1.41344843e+00,  9.71713494e-01,
         1.14103790e+00],
       [ 2.84382529e+00, -6.93077342e-01, -2.63996546e-01,
         6.00204791e-01],
       [-1.96542103e+00,  1.23919915e+00, -2.22778365e-01,
         6.42768062e-01],
       [-1.33574585e-01,  1.67456411e+00, -1.85840701e+00,
         8.49138133e-01],
       [ 3.67151520e-01, -1.59531354e+00,  1.55994725e+00,
         5.18759032e-01],
       [ 3.58409296e+00, -1.34575940e+00, -1.57846651e-01,
      

In [18]:
print(data.shape)
print(reduced_data.shape)

(62, 10)
(62, 4)


In [19]:
print("Pourcentage de variance expliquée : ")
print(sklearn_pca.explained_variance_ratio_)
print("Composantes principales : ")
print(sklearn_pca.components_)

Pourcentage de variance expliquée : 
[0.48017524 0.22078825 0.12605711 0.06488067]
Composantes principales : 
[[ 0.2507328   0.28338618 -0.32111983 -0.29060816 -0.38538159  0.24828001
   0.37336221  0.25423678  0.38127144  0.33119293]
 [ 0.4417748   0.47478054  0.09769085  0.23679916  0.09113044  0.30384009
   0.26672903 -0.42588243 -0.13611297 -0.37700212]
 [ 0.33976579  0.22558359  0.4564193   0.31328208  0.32700136 -0.34542648
   0.0552296   0.40676024  0.19569016  0.30685313]
 [-0.33255618 -0.15648148  0.43623551 -0.1087968   0.34378073  0.64482672
   0.18162367  0.00157754  0.29915565  0.07944389]]
